In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import scape
import katdal
import katpoint
import katsdpscripts
import glob
import re
import pandas as pd 

import csv
import operator

INFO: Generating grammar tables from /usr/lib/python2.7/lib2to3/Grammar.txt
INFO: Generating grammar tables from /usr/lib/python2.7/lib2to3/PatternGrammar.txt


In [3]:
from os import system

In [4]:
print "Scape version: %s"%scape.__version__
print "katdal version: %s"%katdal.__version__
print "katpoint version: %s"%katpoint.__version__
print "katsdpscripts version: %s"%katsdpscripts.__version__

Scape version: 0.1.dev616+master.e836eb2
katdal version: 0.9.6.dev664+master.3ff15ef
katpoint version: 0.8.dev355+master.341001f
katsdpscripts version: 0.1.dev2808+master.790eabe


In [5]:
h5_filename ='/var/kat/archive3/data/MeerKATAR1/telescope_products/2018/04/07/1523065879.h5'
%set_env h5_filename = $h5_filename

env: h5_filename=/var/kat/archive3/data/MeerKATAR1/telescope_products/2018/04/07/1523065879.h5


In [6]:
d = katdal.open(h5_filename);
ant_list,ants = [],[]
[ant_list.append(ant.name) for ant in d.ants]
print(ant_list)

DEBUG: Found int_time=7.996616972 in telstate['sdp_l0_int_time']
DEBUG: Found scale_factor_timestamp=1712000000.0 in telstate[u'i0_scale_factor_timestamp']
DEBUG: Found sync_time=1523010668.0 in telstate[u'i0_sync_time']
DEBUG: Found n_chans=4096 in telstate['sdp_l0_n_chans']
DEBUG: Found bandwidth=856000000.0 in telstate['sdp_l0_bandwidth']
DEBUG: Found center_freq=1284000000.0 in telstate['sdp_l0_center_freq']
DEBUG: Added '1127-145' [radec] (and 1 aliases)
DEBUG: Added '1334-127' [radec] (and 1 aliases)
DEBUG: Added '1345+125' [radec] (and 1 aliases)
DEBUG: Added 'CTD93' [radec] (and 1 aliases)
DEBUG: Added '1709-281' [radec] (and 1 aliases)
DEBUG: Added '1722-554' [radec] (and 1 aliases)
DEBUG: Added 'NRAO530' [radec] (and 1 aliases)
DEBUG: Added '1740-517' [radec] (and 1 aliases)
DEBUG: Added '1827-360' [radec] (and 1 aliases)
DEBUG: Added '1830-210' [radec] (and 1 aliases)
DEBUG: Added 'OV-236' [radec] (and 1 aliases)
DEBUG: Added '1934-638' [radec] (and 1 aliases)
DEBUG: Added '

In [7]:
ant_list.remove('m009') # flatline
ant_list.remove('m022') # bad
ant_list.remove('m004') # bad
ant_list.remove('m025') # died and came back
ant_list.remove('m038') # dead all the time
ant_list.remove('m026') # died during obs

ants=','.join(ant_list)
print ants
%set_env ants=$ants

m000,m001,m002,m003,m005,m006,m008,m010,m011,m012,m013,m014,m015,m016,m017,m020,m021,m023,m024,m027,m029,m030,m031,m032,m033,m034,m036,m039,m040,m041,m042,m043,m044,m045,m046,m047,m048,m049,m050,m051,m052,m053,m054,m055,m056,m057,m058,m059,m060,m061,m062,m063
env: ants=m000,m001,m002,m003,m005,m006,m008,m010,m011,m012,m013,m014,m015,m016,m017,m020,m021,m023,m024,m027,m029,m030,m031,m032,m033,m034,m036,m039,m040,m041,m042,m043,m044,m045,m046,m047,m048,m049,m050,m051,m052,m053,m054,m055,m056,m057,m058,m059,m060,m061,m062,m063


In [8]:
matplotlib.rcParams['font.size']=5.0

In [9]:
ref_ant='m005'
%set_env ref_ant=$ref_ant

env: ref_ant=m005


In [10]:
print "Start H"
res_h = !python /var/kat/katsdpscripts/reduction/baseline_cal_reduction.py -f 2200,2914 -p 'H' -r $ref_ant \
  -a $ants  $h5_filename
print "Done H"

Start H
Done H


In [11]:
print "Start V"
res_v = !python /var/kat/katsdpscripts/reduction/baseline_cal_reduction.py -f 2200,2914 -p 'V' -r $ref_ant \
  -a $ants $h5_filename
print "Done V"

Start V
Done V


In [12]:
delays_h = {}
for i,r in enumerate(res_h):
    if r.startswith('Antenna'):
        ant = r[8:12]
        delays_h[ant] = {}
        delays_h[ant]['E'] = float(res_h[i+2].split(':')[1].lstrip().split(' ')[0])
        delays_h[ant]['N'] = float(res_h[i+3].split(':')[1].lstrip().split(' ')[0])
        delays_h[ant]['U'] = float(res_h[i+4].split(':')[1].lstrip().split(' ')[0])
        delays_h[ant]['D'] = float(res_h[i+5].split(':')[1].lstrip().split(' ')[0])
delays_h

{'m000': {'D': 5875.79, 'E': -8.258, 'N': -207.292, 'U': 1.203},
 'm001': {'D': 5869.962, 'E': 1.125, 'N': -171.763, 'U': 1.053},
 'm002': {'D': 5871.469, 'E': -32.105, 'N': -224.237, 'U': 1.254},
 'm003': {'D': 5872.78, 'E': -66.513, 'N': -202.276, 'U': 0.888},
 'm005': {'D': 5877.82, 'E': -102.088, 'N': -283.12, 'U': 1.475},
 'm006': {'D': 5880.201, 'E': -18.224, 'N': -295.428, 'U': 1.792},
 'm008': {'D': 5874.907, 'E': -93.522, 'N': -535.026, 'U': 3.048},
 'm010': {'D': 5881.439, 'E': 88.1, 'N': -511.871, 'U': 3.774},
 'm011': {'D': 5882.333, 'E': 84.017, 'N': -352.079, 'U': 2.754},
 'm012': {'D': 5863.331, 'E': 140.025, 'N': -368.268, 'U': 3.051},
 'm013': {'D': 5863.477, 'E': 236.801, 'N': -393.463, 'U': 3.719},
 'm014': {'D': 5868.659, 'E': 280.677, 'N': -285.792, 'U': 3.144},
 'm015': {'D': 5916.984, 'E': 210.656, 'N': -219.143, 'U': 2.341},
 'm016': {'D': 5815.573, 'E': 288.161, 'N': -185.881, 'U': 2.433},
 'm017': {'D': 5875.203, 'E': 199.633, 'N': -112.264, 'U': 1.563},
 'm02

In [13]:
delays_v = {}
for i,r in enumerate(res_v):
    if r.startswith('Antenna'):
        ant = r[8:12]
        delays_v[ant] = {}
        delays_v[ant]['E'] = float(res_v[i+2].split(':')[1].lstrip().split(' ')[0])
        delays_v[ant]['N'] = float(res_v[i+3].split(':')[1].lstrip().split(' ')[0])
        delays_v[ant]['U'] = float(res_v[i+4].split(':')[1].lstrip().split(' ')[0])
        delays_v[ant]['D'] = float(res_v[i+5].split(':')[1].lstrip().split(' ')[0])
delays_v

{'m000': {'D': 5877.022, 'E': -8.257, 'N': -207.292, 'U': 1.204},
 'm001': {'D': 5870.972, 'E': 1.126, 'N': -171.763, 'U': 1.054},
 'm002': {'D': 5872.221, 'E': -32.104, 'N': -224.238, 'U': 1.255},
 'm003': {'D': 5874.412, 'E': -66.512, 'N': -202.277, 'U': 0.889},
 'm005': {'D': 5878.919, 'E': -102.088, 'N': -283.12, 'U': 1.475},
 'm006': {'D': 5880.993, 'E': -18.223, 'N': -295.428, 'U': 1.793},
 'm008': {'D': 5875.799, 'E': -93.521, 'N': -535.026, 'U': 3.049},
 'm010': {'D': 5882.954, 'E': 88.101, 'N': -511.872, 'U': 3.775},
 'm011': {'D': 5883.217, 'E': 84.018, 'N': -352.08, 'U': 2.755},
 'm012': {'D': 5864.346, 'E': 140.026, 'N': -368.269, 'U': 3.052},
 'm013': {'D': 5864.225, 'E': 236.802, 'N': -393.463, 'U': 3.72},
 'm014': {'D': 5870.258, 'E': 280.677, 'N': -285.792, 'U': 3.145},
 'm015': {'D': 5918.597, 'E': 210.657, 'N': -219.143, 'U': 2.342},
 'm016': {'D': 5816.715, 'E': 288.161, 'N': -185.881, 'U': 2.432},
 'm017': {'D': 5875.953, 'E': 199.633, 'N': -112.264, 'U': 1.563},
 '

# Cross check between polarizations for derived positions

In [14]:
for a in sorted(delays_v):
    print a,
    print("diff E %06.3f") %(delays_h[a]['E'] - delays_v[a]['E']),
    print("diff N %06.3f") %(delays_h[a]['N'] - delays_v[a]['N']),
    print("diff U %06.3f") %(delays_h[a]['U'] - delays_v[a]['U']),
    print("diff D %06.3f") %(delays_h[a]['D'] - delays_v[a]['D'])
    
# expect differences between E,N and U less than 1cm and Delays less than 3m 

m000 diff E -0.001 diff N 00.000 diff U -0.001 diff D -1.232
m001 diff E -0.001 diff N 00.000 diff U -0.001 diff D -1.010
m002 diff E -0.001 diff N 00.001 diff U -0.001 diff D -0.752
m003 diff E -0.001 diff N 00.001 diff U -0.001 diff D -1.632
m005 diff E 00.000 diff N 00.000 diff U 00.000 diff D -1.099
m006 diff E -0.001 diff N 00.000 diff U -0.001 diff D -0.792
m008 diff E -0.001 diff N 00.000 diff U -0.001 diff D -0.892
m010 diff E -0.001 diff N 00.001 diff U -0.001 diff D -1.515
m011 diff E -0.001 diff N 00.001 diff U -0.001 diff D -0.884
m012 diff E -0.001 diff N 00.001 diff U -0.001 diff D -1.015
m013 diff E -0.001 diff N 00.000 diff U -0.001 diff D -0.748
m014 diff E 00.000 diff N 00.000 diff U -0.001 diff D -1.599
m015 diff E -0.001 diff N 00.000 diff U -0.001 diff D -1.613
m016 diff E 00.000 diff N 00.000 diff U 00.001 diff D -1.142
m017 diff E 00.000 diff N 00.000 diff U 00.000 diff D -0.750
m020 diff E 00.000 diff N 00.000 diff U 00.001 diff D -1.142
m021 diff E 00.001 diff 

In [15]:
# write solutions to file
matchRe = re.findall('[0-9]{10,}', h5_filename)
filename='delay_file_'+''.join(matchRe)+'.txt'

with open(filename, 'w') as f:
    for h,v in zip(delays_h,delays_v):
        print >> f, h,v, mean([delays_h[h]['E'],delays_v[v]['E']]) , \
                    mean([delays_h[h]['N'],delays_v[v]['N']]), \
                    mean([delays_h[h]['U'],delays_v[v]['U']]), \
                    delays_h[h]['D'],delays_v[v]['D']
    f.close()

In [16]:
# sort delay file by antenna name and load antennaLOC
sample = open(filename, 'r')
sorter = csv.reader(sample, delimiter=' ')
sort = sorted(sorter, key=operator.itemgetter(1))

with open('ant_loc.csv', 'r') as f:
    tocs = f.read()

In [17]:
cols = ['Ants', 'Delay', 'East', 'North', 'UP', 'TOC']
df = pd.DataFrame(columns=cols)
for line in sort:
    p = re.compile(line[0]+',\s+\d{4}\.\d{1,3}')
    toc = float(p.search(tocs).group()[6:])
    df = df.append(pd.Series(
                             [
                                line[0],
                                line[-2],
                                line[2],
                                line[3],
                                line[4], 
                                toc
                             ], 
                              index=['Ants', 'Delay', 'East', 'North', 'UP', 'TOC']
                            ), ignore_index = True)

In [18]:
deltaTOC, deltaUP = [], []
deltaTOC.append(0)
deltaUP.append(0)
for i in range(1, len(df.TOC[1:])+1):
    deltaTOC.append(df.TOC[i] - df.TOC[i-1])
    deltaUP.append(float(df.UP[i]) - float(df.UP[i-1]))

em  = [' ' for i in range(len(deltaUP))]
df[' '] =  pd.Series(em,  index = df.index)

df['delta UP']  = pd.Series(deltaUP,  index = df.index)
df['delta TOC'] = pd.Series(deltaTOC, index = df.index)

print('\nDelay Solutions and antenna locations\n')
df


Delay Solutions and antenna locations



,Ants,Delay,East,North,UP,TOC,,delta UP,delta TOC
0,m000,5875.79,-8.2575,-207.292,1.2035,1052.205,,0.0000,0.000
1,m001,5869.962,1.1255,-171.763,1.0535,1051.874,,-0.1500,-0.331
2,m002,5871.469,-32.1045,-224.2375,1.2545,1052.210,,0.2010,0.336
3,m003,5872.78,-66.5125,-202.2765,0.8885,1051.754,,-0.3660,-0.456
4,m005,5877.82,-102.088,-283.12,1.475,1052.430,,0.5865,0.676
5,m006,5880.201,-18.2235,-295.428,1.7925,1052.590,,0.3175,0.160
6,m008,5874.907,-93.5215,-535.026,3.0485,1053.990,,1.2560,1.400
7,m010,5881.439,88.1005,-511.8715,3.7745,1054.756,,0.7260,0.766
8,m011,5882.333,84.0175,-352.0795,2.7545,1053.521,,-1.0200,-1.235
9,m012,5863.331,140.0255,-368.2685,3.0515,1053.730,,0.2970,0.209


In [19]:
# send file to ops server to process and push to github

opsServer='ops.kat.ac.za'
rPath='/home/kat/ops_team_sw/utilities/update_delay_models'
cmd = "sshpass -p {} scp -r {} kat@{}:{}".format('kat', filename, opsServer, rPath)
system(cmd)


32512

In [38]:
for x,y in enumerate(res_h):
    print res_h[x][:65]


Loading and processing data...

DEBUG: Found int_time=7.996616972 in telstate['sdp_l0_int_time']
DEBUG: Found scale_factor_timestamp=1712000000.0 in telstate[u'i0
DEBUG: Found sync_time=1523010668.0 in telstate[u'i0_sync_time']
DEBUG: Found n_chans=4096 in telstate['sdp_l0_n_chans']
DEBUG: Found bandwidth=856000000.0 in telstate['sdp_l0_bandwidth'
DEBUG: Found center_freq=1284000000.0 in telstate['sdp_l0_center_
antennas (52): m000, m001, m002, m003, m005 (*ref*), m006, m008, 
baselines (1326): 0-1 0-2 1-2 0-3 1-3 2-3 0-4 1-4 2-4 3-4 0-5 1-5
scan   0 (   9 samples) skipped 'slew'
scan   1 (  16 samples) 0.032 0.034 0.034 0.034 0.034 0.035 0.036
scan   2 (  22 samples) skipped 'slew'
scan   3 (  16 samples) 0.056 0.065 0.061 0.057 0.053 0.062 0.060
scan   4 (   4 samples) skipped 'slew'
scan   5 (  16 samples) 0.031 0.033 0.030 0.032 0.031 0.034 0.031
scan   6 (   4 samples) skipped 'slew'
scan   7 (  16 samples) 0.032 0.034 0.033 0.033 0.032 0.036 0.036
scan   8 (   7 samples) skipped

In [37]:
for x,y in enumerate(res_v):
    print res_v[x][:65]


Loading and processing data...

DEBUG: Found int_time=7.996616972 in telstate['sdp_l0_int_time']
DEBUG: Found scale_factor_timestamp=1712000000.0 in telstate[u'i0
DEBUG: Found sync_time=1523010668.0 in telstate[u'i0_sync_time']
DEBUG: Found n_chans=4096 in telstate['sdp_l0_n_chans']
DEBUG: Found bandwidth=856000000.0 in telstate['sdp_l0_bandwidth'
DEBUG: Found center_freq=1284000000.0 in telstate['sdp_l0_center_
antennas (52): m000, m001, m002, m003, m005 (*ref*), m006, m008, 
baselines (1326): 0-1 0-2 1-2 0-3 1-3 2-3 0-4 1-4 2-4 3-4 0-5 1-5
scan   0 (   9 samples) skipped 'slew'
scan   1 (  16 samples) 0.037 0.037 0.039 0.037 0.039 0.038 0.040
scan   2 (  22 samples) skipped 'slew'
scan   3 (  16 samples) 0.066 0.071 0.071 0.063 0.061 0.067 0.068
scan   4 (   4 samples) skipped 'slew'
scan   5 (  16 samples) 0.037 0.036 0.035 0.035 0.036 0.037 0.035
scan   6 (   4 samples) skipped 'slew'
scan   7 (  16 samples) 0.038 0.037 0.038 0.036 0.037 0.038 0.039
scan   8 (   7 samples) skipped